## CLIP LOADING AND INFERENCE TESTING:

In [6]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import time
import urllib

print(f"TORCH VERSION: {torch.__version__}")
print(f"CUDA AVAILABLE: {torch.cuda.is_available()}")

TORCH VERSION: 2.9.1+cpu
CUDA AVAILABLE: False


In [7]:
# LOAD CLIP MODEL

print("LOADING CLIP MODEL")
start_time = time.time()
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
load_time = time.time() - start_time

print(f"CLIP MODEL LOADED IN {load_time:.2f} SECONDS")
print(f"MODEL DEVICE: {next(model.parameters()).device}")




LOADING CLIP MODEL
CLIP MODEL LOADED IN 1.88 SECONDS
MODEL DEVICE: cpu


In [8]:
# TEST CLIP INFERENCE ON SINGLE IMAGE-TEXT PAIR

# DOWNLOAD IMAGE
image_path = "img/cat_image.jpg" # EXAMPLE SINGLE IMAGE TAKEN FROM WIKIPEDIA (REAL LABEL = CAT)

# LOAD IMAGE
image = Image.open(image_path).convert("RGB")
image.thumbnail((224, 224)) # RESIZE FOR SPEED 

# INFERENCE RUN

print("RUNNING INFERENCE ON CLIP MODEL")
texts = ["a photo of a cat", "a photo of a dog", "a photo of a car"]  # EXAMPLE TEXTS
inputs = processor(text = texts, images = image, return_tensors = "pt", padding = True)
inputs = {k: v.to(device) for k, v in inputs.items()}

# TIME METRICS
start = time.time()
outputs = model(**inputs)
inference_time = time.time() - start

logits_per_image = outputs.logits_per_image  # THIS IS THE IMAGE-TO-TEXT SIMILARITY SCORE
probs = logits_per_image.softmax(dim=1)  # WE CAN TAKE THE SOFTMAX TO GET THE PROBABILITIES OF EACH TEXT

print(f"INFERENCE COMPLETED IN {inference_time:.2f} SECONDS")
print(f"\nSCORES:")
for text, prob in zip(texts, probs[0]):
    print(f"'{text}': {prob.item():.4f}")


RUNNING INFERENCE ON CLIP MODEL
INFERENCE COMPLETED IN 0.12 SECONDS

SCORES:
'a photo of a cat': 0.9973
'a photo of a dog': 0.0027
'a photo of a car': 0.0001


## LLaVA LOADING AND INFERENCE TESTING:


In [9]:
from transformers import AutoProcessor, LlavaForConditionalGeneration

print("LOADING LLavA MODEL")

start_time = time.time()
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
model = LlavaForConditionalGeneration.from_pretrained(
    "llava-hf/llava-1.5-7b-hf",  # MODEL NAME
    torch_dtype=torch.float16, # USE FP16 FOR SPEED
    device_map="auto", # AUTOMATICALLY PLACE ON GPU IF AVAILABLE
)

load_time = time.time() - start_time

print(f"LLaVA MODEL LOADED IN {load_time:.2f} SECONDS")

LOADING LLavA MODEL


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 30.87it/s]
Some parameters are on the meta device because they were offloaded to the cpu and disk.


LLaVA MODEL LOADED IN 2.92 SECONDS


In [10]:
from PIL import Image
import torch
import time
from transformers import AutoProcessor

print("TESTING LLaVA INFERENCE (STANDARD AUTO-PROCESSOR)")

# LOAD IMAGE / PROCESSOR
image = Image.open(image_path).convert("RGB")
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

# DEFINE PROMPT
prompt = "USER: <image>\nDescribe this image in detail.\nASSISTANT:"

# INPUTS
inputs = processor(text=prompt, images=image, return_tensors="pt").to(model.device)

# INFERENCE RUN
print(f"\nPROMPT: '{prompt}'")
print(f"GENERATING RESPONSE...")

start = time.time()
with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False
    )
inference_time = time.time() - start

# DECODE RESPONSE
response = processor.decode(output_ids[0], skip_special_tokens=True)

print(f"\n INFERENCE COMPLETED IN {inference_time:.2f} SECONDS")
print(f"\n LLaVA Response:\n{response}")


TESTING LLaVA INFERENCE (STANDARD AUTO-PROCESSOR)

PROMPT: 'USER: <image>
Describe this image in detail.
ASSISTANT:'
GENERATING RESPONSE...

 INFERENCE COMPLETED IN 2700.31 SECONDS

 LLaVA Response:
USER:  
Describe this image in detail.
ASSISTANT: The image features a cat sitting on a branch, possibly a tree branch. The cat is looking at the camera, displaying a curious expression. The cat is positioned in the center of the image, with its body facing the viewer. The scene appears to be outdoors, with the cat enjoying its time in the natural environment.
